### Interpolação e diferenciação numérica (cap 4)

---

1. [Polinômio interpolador](#Polinômio-interpolador)
    * [Matriz de Vandermonde](#Matriz-de-Vandermonde)
    * [Exercício 1](#Exercício-1:)
    * [Polinômio de Lagrange](#Polinômio-de-Lagrange)
    * [Diferenças divididas](#Diferenças-divididas)

2. [xxx](#Método-da-bisseção)
3. [xxx](#Método-de-Newton)

## 28-08

### Polinômio interpolador (4.1)

Dado um conjunto de $n$ pontos $(x, y)$, o polinômio interpolador é um polinômio de grau $n-1$ que passa por todos os pontos da forma:

$f(x_{i}) = a_{0} + a_{1}x_{i} + a_{2}x_{i}^{2} + ... + a_{n}x_{i}^{n} = y_{i}$, para $i \in [1,n]$

Dadas as $n$ equações, temos um sistema linear da forma $Ac = y$, no qual queremos obter o vetor $c$ dos coeficientes da equação.

#### Matriz de Vandermonde

É a matriz $A$, com o primeiro elemento de cada linha $A_i$ igual a 1 e os demais elementos sendo as $n-1$ potências de $x_i$

In [66]:
np.vander([1,2,3])

array([[1, 1, 1],
       [4, 2, 1],
       [9, 3, 1]])

#### Exercício 1:

Determine o polinômio de grau 2 que passa por $P_0 = (0,-1)$, $P_1 = (1,1)$, $P_2(2,5)$.

In [67]:
import numpy as np

grau = 2
pontos = [(0,-1), (1,1), (2,5)]
x = [0,0,0]

In [68]:
def Jacobi(A, b, x, threshold=1000):
    
    """
    Resolove o sistema linear pelo método de Jacobi.
    
    :param A: matriz nxn
    :param b: vetor nx1
    :param x: vetor inicial
    :param threshold: número máximo de iterações
    
    :returns: vetor solução do sistema, número de iterações
    """
    
    U = np.triu(A)
    np.fill_diagonal(U, 0)
    
    L = np.tril(A)
    np.fill_diagonal(L, 0)
    
    D = np.diag(np.diag(A))
    D_inv = np.linalg.inv(D) 
    
    t = 0
    while t < threshold:
        
        x = np.inner(D_inv, b - np.inner((L+U), x))
        t += 1
    
    return x, t

In [78]:
def interpol_pol(n, pontos, x):
    
    """
    Resolove o sistema linear pelo método de Jacobi.
    
    :param n: grau do polinômio
    :param pontos: lista com n+1 pontos (x,y)
    :param x: vetor inicial nx1
    :param threshold: número máximo de iterações
    
    :returns: coeficientes do polinômio, número de iterações
    """
    
    b = [i[1] for i in pontos]
    M = np.vander([i[0] for i in pontos], increasing=True)
    
    x, t = Jacobi(M, b, x)
    
    return x, t

In [79]:
interpol_pol(grau, pontos, x)

(array([-1.,  1.,  1.]), 1000)

#### [Polinômio de Lagrange](https://pt.wikipedia.org/wiki/Polin%C3%B4mio_de_Lagrange)

Esse é outro método utilizado para a solução de polinômios interpoladores. 

O polinômio de Lagrange é da forma: 
$L(x) := \sum_{j=0}^{k} y_j l_j(x)$

Sendo $l_{j}(x):=\prod _{i=0,j\neq i}^{k}{\frac {x-x_{i}}{x_{j}-x_{i}}}={\frac {x-x_{0}}{x_{j}-x_{0}}}\cdots {\frac {x-x_{j-1}}{x_{j}-x_{j-1}}}{\frac {x-x_{j+1}}{x_{j}-x_{j+1}}}\cdots {\frac {x-x_{k}}{x_{j}-x_{k}}}$

#### Diferenças divididas

Outro método é calcularmos a interpolação recursivamente, adicionando um grau a cada ponto avaliado.

Tomando como exemplo o Exercício 1, escolhemos um dos pontos $(x, y)$ como inicial e começamos com um polinômio de grau $0$:

$p_0(x_0) = c_0$

$(x_0, y_0) = (0, -1) \rightarrow c_0 = -1$

Ao avaliarmos o próximo ponto, construimos o polinômio $p_1$ de grau $1$ a partir de $p_0$. 

Adicionamos um novo termo $c_{1}(x - x_0)$, essa diferença é necessária já que queremos que o novo polinômio interpole $(x_0, y_0)$.

$p_1(x) = p_0(x) + c_{1}(x - 0)$

$(x_1, y_1) = (1, 1) \rightarrow p_1(x_1) = -1 + c_{1} = 1 \rightarrow c_1 = 2$

Por último, adicionamos o termo de grau $2$: $c_{2}(x - x_{1})(x - x_{0})$, na mesma lógica de antes.

$p_2(x) = p_1(x) + c_{2}(x - x_{1})(x - x_{0})$

$(x_2, y_2) = (2, 5) \rightarrow p_2(x_2) = -1 + c_{1}2 + c_{2}(2 - 1)(2 - 0) = 5 \rightarrow c_2 = 1$

Logo, obtemos o polinômio $P(x) = p_2(x) = -1 + 2x + x(x-1)$